In [37]:
from transformers import AutoFeatureExtractor, CvtForImageClassification, AutoTokenizer, RobertaModel
from torch.utils.data import Dataset, DataLoader
from string import digits
from html import unescape
from sklearn.model_selection import train_test_split
from torchmetrics.functional.classification import auroc, accuracy
from lightning.pytorch.callbacks import ModelCheckpoint

import torchvision
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import pickle
from lightning.pytorch import seed_everything
import lightning.pytorch as lp

torch.set_float32_matmul_precision('medium')

In [2]:
## os.chdir(r'C:\Users\rabby\CS 7643 - Deep Learning\Project')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = {
    'vmodel_name': 'microsoft/cvt-21',
    'vmodel_path': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Unimodal Models\Finetuned_CvT_589.pth.tar',
    'tmodel_name': 'roberta-base',
    'tmodel_path': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Unimodal Models\Finetuned_Roberta_Lou.pth.tar',
    'n_labels': 2,
    'batch_size': 32,
    'dropout': 0.492,
    'w_decay': 0.032,
    'lr': 5e-5,
    'hidden_size': 128,
    'n_epochs': 15,
    'device': device,
    'n_img_train': 1,
    'n_img_val': 1,
    'n_img_test': 1000,
    'img_train': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Data\img_preprocessed',
    'img_val': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Data\img_preprocessed',
    'img_test': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Data\img_preprocessed',
    'label_train':r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Data\hateful_memes\train.jsonl',
    'label_val': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Data\hateful_memes\dev_seen.jsonl',
    'label_test': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Data\hateful_memes\test_seen.jsonl',
    'reload': False,
    'img_train_rl': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Data\hateful_memes\img_train.pt',
    'img_val_rl': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Data\hateful_memes\img_val.pt',
    'img_test_rl': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Data\hateful_memes\img_test.pt',
    'text_train_rl': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Data\hateful_memes\text_train.pkl',
    'text_val_rl': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Data\hateful_memes\text_val.pkl',
    'text_test_rl': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Data\hateful_memes\text_test.pkl',
    'label_train_rl': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Data\hateful_memes\labels_train.pkl',
    'label_val_rl': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Data\hateful_memes\labels_val.pkl',
    'label_test_rl': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Data\hateful_memes\labels_test.pkl',
    'seed': 24,
    'ckpt_path': r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Unimodal Models',
    'val_every': 1
}

In [3]:
def Load_Memes(config):
    #Load Extractor
    extractor = AutoFeatureExtractor.from_pretrained(config['vmodel_name'])
    #Load Train Data
    n = 0
    label_train = pd.read_json(config['label_train'], lines = True, nrows = config['n_img_train'])
    img_train_id = list(label_train['id'].values)
    img_train_list = [config['img_train'] + r'\\' + str(i).zfill(5) + '.png' for i in img_train_id]
    for img_path in img_train_list:
        if n == 0:
            img_train = torch.tensor(extractor(torchvision.io.read_image(img_path), return_tensors="pt")['pixel_values'][0])
            print('Training Image', str(n), 'loaded.')
        else:
            img_train = torch.cat((img_train.view(n, 3, 224, 224), torch.tensor(extractor(torchvision.io.read_image(img_path), return_tensors="pt")['pixel_values'][0]).view(1, 3, 224, 224)))
        n += 1
        if (n % 1000 == 0):
            print('Training Image', str(n), 'loaded.')
    
    text_train = list(label_train['text'].values)
    print('Training Text', str(config['n_img_train']), 'loaded.')
    
    labels_train = list(label_train['label'].values)
    print('Training Label', str(config['n_img_train']), 'loaded.')
    
    n = 0
    label_val = pd.read_json(config['label_val'], lines = True, nrows = config['n_img_val'])
    img_val_id = list(label_val['id'].values)
    img_val_list = [config['img_val'] + r'\\' + str(i).zfill(5) + '.png' for i in img_val_id]
    for img_path in img_val_list:
        if n == 0:
            img_val = torch.tensor(extractor(torchvision.io.read_image(img_path), return_tensors="pt")['pixel_values'][0])
            print('Validation Image', str(n), 'loaded.')
        else:
            img_val = torch.cat((img_val.view(n, 3, 224, 224), torch.tensor(extractor(torchvision.io.read_image(img_path), return_tensors="pt")['pixel_values'][0]).view(1, 3, 224, 224)))
        n += 1
        if (n % 1000 == 0):
            print('Validation Image', str(n), 'loaded.')
    
    text_val = list(label_val['text'].values)
    print('Validation Text', str(config['n_img_val']), 'loaded.')
    
    labels_val = list(label_val['label'].values)
    print('Validation Label', str(config['n_img_val']), 'loaded.')
    
    n = 0
    label_test = pd.read_json(config['label_test'], lines = True, nrows = config['n_img_test'])
    img_test_id = list(label_test['id'].values)
    img_test_list = [config['img_test'] + r'\\' + str(i).zfill(5) + '.png' for i in img_test_id]
    for img_path in img_test_list:
        if n == 0:
            img_test = torch.tensor(extractor(torchvision.io.read_image(img_path), return_tensors="pt")['pixel_values'][0])
            print('Test Image', str(n), 'loaded.')
        else:
            img_test = torch.cat((img_test.view(n, 3, 224, 224), torch.tensor(extractor(torchvision.io.read_image(img_path), return_tensors="pt")['pixel_values'][0]).view(1, 3, 224, 224)))
        n += 1
        if (n % 1000 == 0):
            print('Test Image', str(n), 'loaded.')
    
    text_test = list(label_test['text'].values)
    print('Test Text', str(config['n_img_test']), 'loaded.')
    
    labels_test = list(label_test['label'].values)
    print('Test Label', str(config['n_img_test']), 'loaded.')
    
    return img_train, text_train, labels_train, img_val, text_val, labels_val, img_test, text_test, labels_test

In [4]:
class Memes_Dataset(Dataset):
    
    def __init__(self, image, text, label, tokenizer, max_length = 48):
        #Declare variables
        self.image = image
        self.text = text
        self.labels = torch.tensor(label)
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        meme_text = self.text[index]
        tokens = self.tokenizer.encode_plus(meme_text, add_special_tokens = True, return_tensors = 'pt', truncation = True, 
                                           max_length = self.max_length, padding = 'max_length', return_attention_mask = True)
        
        return {'image': self.image[index], 'input_ids': tokens.input_ids.flatten(), 
                'attention_mask': tokens.attention_mask.flatten(), 'labels': self.labels[index]}
    
class Memes_Data_Module(lp.LightningDataModule):
    
    def __init__(self, train_image, train_text, train_labels, 
                 val_image, val_text, val_labels, 
                 test_image = None, test_text = None, test_labels = None, 
                 batch_size = 32, max_length = 48, text_model = 'roberta-base'):
        super().__init__()
        self.train_img = train_image
        self.train_text = train_text
        self.train_labels = train_labels
        self.val_img = val_image
        self.val_text = val_text
        self.val_labels = val_labels

        if test_image == None:
            self.test_img = val_image
            self.test_text = val_text
            self.test_labels = val_labels
        else:
            self.test_img = test_image
            self.test_text = test_text
            self.test_labels = test_labels
        self.batch_size = batch_size
        self.max_length = max_length
        self.tokenizer = AutoTokenizer.from_pretrained(text_model)
        
    def setup(self, stage = None):
        self.train_ds = Memes_Dataset(self.train_img, self.train_text, self.train_labels, 
                                      self.tokenizer, max_length = self.max_length)
        self.val_ds = Memes_Dataset(self.val_img, self.val_text, self.val_labels, 
                                    self.tokenizer, max_length = self.max_length)
        self.test_ds = Memes_Dataset(self.test_img, self.test_text, self.test_labels,
                                    self.tokenizer, max_length = self.max_length)
        
    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size = self.batch_size, shuffle = True)
    
    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size = self.batch_size, shuffle = False)
    
    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size = self.batch_size, shuffle = False)

In [5]:
class CVT_Fairface(pl.LightningModule):

    def __init__(self, config):
        super().__init__()
        self.config = config
        self.feature_extractor = AutoFeatureExtractor.from_pretrained(config['model_name'])
        self.model = CvtForImageClassification.from_pretrained(config['model_name']).to(device)
        self.new_classifier = nn.Linear(384, self.config['n_labels']).to(device)
        torch.nn.init.xavier_uniform_(self.new_classifier.weight)
        self.model.classifier = self.new_classifier
        self.softmax = nn.Softmax(dim = 1)
        self.loss = nn.CrossEntropyLoss()
        self.training_step_outputs = []
        self.training_auroc = []
        self.training_acc = []
        self.validation_step_outputs = []
        self.validation_auroc = []
        self.validation_acc = []
        self.test_step_outputs = []
        self.test_auroc = []
        self.test_acc = []
        self.tloss = []
        self.tauroc = []
        self.tacc = []
        self.vloss = []
        self.vauroc = []
        self.vacc = []
           
    def forward(self, x, labels = None):
        features = x.to(self.config['device'])
        out = self.model(features)
        return out.logits
    
    def training_step(self, batch, batch_index):
        loss, out, y = self._common_step(batch, batch_index)
        pred = self.softmax(out)
        t_auroc = auroc(pred, y, task = 'multiclass', num_classes = self.config['n_labels'])
        t_acc = accuracy(pred, y, task = 'multiclass', num_classes = self.config['n_labels'])
        self.training_step_outputs.append(loss)
        self.training_auroc.append(t_auroc)
        self.training_acc.append(t_acc)
        self.log("Training Accuracy", t_acc, prog_bar = True, logger = True)
        return loss
    
    def on_train_epoch_end(self):
        epoch_mean = torch.stack(self.training_step_outputs).mean()
        epoch_auroc = torch.stack(self.training_auroc).mean()
        epoch_acc = torch.stack(self.training_acc).mean()
        self.tloss.append(float(epoch_mean.detach().cpu().numpy()))
        self.tauroc.append(float(epoch_auroc.detach().cpu().numpy()))
        self.tacc.append(float(epoch_acc.detach().cpu().numpy()))
        self.training_step_outputs.clear()
        self.training_auroc.clear()
        self.training_acc.clear()
    
    def validation_step(self, batch, batch_index):
        loss, out, y = self._common_step(batch, batch_index)
        pred = self.softmax(out)
        v_auroc = auroc(pred, y, task = 'multiclass', num_classes = self.config['n_labels'])
        v_acc = accuracy(pred, y, task = 'multiclass', num_classes = self.config['n_labels'])
        self.validation_step_outputs.append(loss)
        self.validation_auroc.append(v_auroc)
        self.validation_acc.append(v_acc)
        self.log("Validation Accuracy", v_acc, prog_bar = True, logger = True)
        return loss
    
    def on_validation_epoch_end(self):
        epoch_mean = torch.stack(self.validation_step_outputs).mean()
        epoch_auroc = torch.stack(self.validation_auroc).mean()
        epoch_acc = torch.stack(self.validation_acc).mean()
        self.vloss.append(float(epoch_mean.detach().cpu().numpy()))
        self.vauroc.append(float(epoch_auroc.detach().cpu().numpy()))
        self.vacc.append(float(epoch_acc.detach().cpu().numpy()))
        self.validation_step_outputs.clear()
        self.validation_auroc.clear()
        self.validation_acc.clear()
    
    def test_step(self, batch, batch_index):
        loss, out, y = self._common_step(batch, batch_index)
        pred = self.softmax(out)
        t_auroc = auroc(pred, y, task = 'multiclass', num_classes = self.config['n_labels'])
        t_acc = accuracy(pred, y, task = 'multiclass', num_classes = self.config['n_labels'])
        self.test_step_outputs.append(loss)
        self.test_auroc.append(t_auroc)
        self.test_acc.append(t_acc)
        self.log("Test Accuracy", t_acc, prog_bar = True, logger = True)
        return loss

    def predict_step(self, batch, batch_index):
        loss, out, y = self._common_step(batch, batch_index)
        return loss
    
    def _common_step(self, batch, batch_index):
        x, y = batch
        x = x.type(torch.cuda.FloatTensor)
        y = y.type(torch.LongTensor)
        y = y.to(config['device'])
        out = self.forward(x)
        loss = self.loss(out, y)
        return loss, out, y
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr = self.config['lr'])
        return [optimizer]
    
    def plot_loss(self):
        self.vloss.pop()
        plt.plot(self.tloss, label = 'Training')
        plt.plot(self.vloss, label = 'Validation')
        plt.title('Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()
        
    def plot_auroc(self):
        self.vauroc.pop()
        plt.plot(self.tauroc, label = 'Training')
        plt.plot(self.vauroc, label = 'Validation')
        plt.title('AUROC')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()
        
    def plot_accuracy(self):
        self.vacc.pop()
        plt.plot(self.tacc, label = 'Training')
        plt.plot(self.vacc, label = 'Validation')
        plt.title('Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

In [6]:
class Roberta_Pol(pl.LightningModule):

    def __init__(self, config):
        super().__init__()
        self.config = config
        self.model = AutoModel.from_pretrained(config['model_name'], return_dict = True)
        self.classifier = nn.Linear(self.model.config.hidden_size, self.config['n_labels'])
        self.softmax = nn.Softmax(dim = 1)
        self.loss = nn.MSELoss()
        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.tloss = []
        self.vloss = []
           
    def forward(self, input_ids, attention_mask, labels = None):
        out = self.model(input_ids = input_ids, attention_mask = attention_mask)
        out = torch.mean(out.last_hidden_state, 1)
        # final logits
        out = self.classifier(out)
        return out
    
    def training_step(self, batch, batch_index):
        loss, out, y = self._common_step(batch, batch_index)
        self.training_step_outputs.append(loss)
        self.log("Training Loss", loss, prog_bar = True, logger = True)
        return loss
    
    def on_train_epoch_end(self):
        epoch_mean = torch.stack(self.training_step_outputs).mean()
        self.tloss.append(float(epoch_mean.detach().cpu().numpy()))
        self.training_step_outputs.clear()
    
    def validation_step(self, batch, batch_index):
        loss, out, y = self._common_step(batch, batch_index)
        self.validation_step_outputs.append(loss)
        self.log("Validation Loss", loss, prog_bar = True, logger = True)
        return loss
    
    def on_validation_epoch_end(self):
        epoch_mean = torch.stack(self.validation_step_outputs).mean()
        self.vloss.append(float(epoch_mean.detach().cpu().numpy()))
        self.validation_step_outputs.clear()
    
    def test_step(self, batch, batch_index):
        loss, out, y = self._common_step(batch, batch_index)
        self.log("Test Loss", loss, prog_bar = True, logger = True)
        return loss

    def predict_step(self, batch, batch_index):
        loss, out, y = self._common_step(batch, batch_index)
        return loss
    
    def _common_step(self, batch, batch_index):
        x = batch['input_ids']
        y = batch['labels'].squeeze(1)
        attn_mask = batch['attention_mask']
        out = self.forward(x, attn_mask)
        y = y.to(config['device'])
        loss = self.loss(out, y)
        return loss, out, y
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr = self.config['lr'])
        return [optimizer]
    
    def plot_loss(self):
        self.vloss.pop()
        plt.plot(self.tloss, label = 'Training')
        plt.plot(self.vloss, label = 'Validation')
        plt.title('Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()
        
    def plot_auroc(self):
        self.vauroc.pop()
        plt.plot(self.tauroc, label = 'Training')
        plt.plot(self.vauroc, label = 'Validation')
        plt.title('AUROC')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()
        
    def plot_accuracy(self):
        self.vacc.pop()
        plt.plot(self.tacc, label = 'Training')
        plt.plot(self.vacc, label = 'Validation')
        plt.title('Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

In [7]:
class CVT_Roberta(lp.LightningModule):
    
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.cvt = torch.load(config['vmodel_path'])
        self.roberta = torch.load(config['tmodel_path'])
        self.layernorm_1 = nn.LayerNorm(self.config['hidden_size'])
        self.layernorm_2 = nn.LayerNorm(self.config['hidden_size'])
        self.batchnorm_1 = nn.BatchNorm1d(self.config['hidden_size'])
        self.batchnorm_2 = nn.BatchNorm1d(self.config['hidden_size'])
        self.linear_cvt = nn.Linear(384, self.config['hidden_size'])
        self.linear_roberta = nn.Linear(self.roberta.model.config.hidden_size, self.config['hidden_size'])
        self.classifier = nn.Sequential(nn.Linear(2 * self.config['hidden_size'], 2 * self.config['hidden_size']),
                                       nn.ReLU(),
                                       nn.Dropout(config['dropout']),
                                        nn.LayerNorm(2 * self.config['hidden_size']),
                                        nn.BatchNorm1d(2 * self.config['hidden_size']),
                                       nn.Linear(2 * self.config['hidden_size'], self.config['n_labels']))
        self.cvt.model.classifier = self.linear_cvt
        self.roberta.classifier = self.linear_roberta
        self.softmax = nn.Softmax(dim = 1)
        self.loss = nn.CrossEntropyLoss()
        self.training_step_outputs = []
        self.training_auroc = []
        self.training_acc = []
        self.validation_step_outputs = []
        self.validation_auroc = []
        self.validation_acc = []
        self.test_step_outputs = []
        self.test_auroc = []
        self.test_acc = []
        self.tloss = []
        self.tauroc = []
        self.tacc = []
        self.vloss = []
        self.vauroc = []
        self.vacc = []
        
    def forward(self, img, text, attn_mask, labels = None):
        img = torch.squeeze(img)
        v_out = self.batchnorm_1(self.layernorm_1(self.cvt(img)))
        t_out = self.batchnorm_2(self.layernorm_2(self.roberta(text, attn_mask)))
        out = torch.cat((v_out, t_out), dim = 1)
        out = self.classifier(out)
        return out
    
    def training_step(self, batch, batch_index):
        loss, out, y = self._common_step(batch, batch_index)
        pred = self.softmax(out)
        t_auroc = auroc(pred, y, task = 'multiclass', num_classes = self.config['n_labels'])
        t_acc = accuracy(pred, y, task = 'multiclass', num_classes = self.config['n_labels'])
        self.training_step_outputs.append(loss)
        self.training_auroc.append(t_auroc)
        self.training_acc.append(t_acc)
        self.log("Training Accuracy", t_acc, prog_bar = True, logger = True)
        return loss
    
    def on_train_epoch_end(self):
        epoch_mean = torch.stack(self.training_step_outputs).mean()
        epoch_auroc = torch.stack(self.training_auroc).mean()
        epoch_acc = torch.stack(self.training_acc).mean()
        self.tloss.append(round(float(epoch_mean.detach().cpu().numpy()), 4))
        self.tauroc.append(round(float(epoch_auroc.detach().cpu().numpy()), 4))
        self.tacc.append(round(float(epoch_acc.detach().cpu().numpy()), 4))
        self.training_step_outputs.clear()
        self.training_auroc.clear()
        self.training_acc.clear()
    
    def validation_step(self, batch, batch_index):
        loss, out, y = self._common_step(batch, batch_index)
        pred = self.softmax(out)
        v_auroc = auroc(pred, y, task = 'multiclass', num_classes = self.config['n_labels'])
        v_acc = accuracy(pred, y, task = 'multiclass', num_classes = self.config['n_labels'])
        self.validation_step_outputs.append(loss)
        self.validation_auroc.append(v_auroc)
        self.validation_acc.append(v_acc)
        self.log("Validation Accuracy", v_acc, prog_bar = True, logger = True)
        return loss
    
    def on_validation_epoch_end(self):
        epoch_mean = torch.stack(self.validation_step_outputs).mean()
        epoch_auroc = torch.stack(self.validation_auroc).mean()
        epoch_acc = torch.stack(self.validation_acc).mean()
        self.vloss.append(round(float(epoch_mean.detach().cpu().numpy()), 4))
        self.vauroc.append(round(float(epoch_auroc.detach().cpu().numpy()), 4))
        self.vacc.append(round(float(epoch_acc.detach().cpu().numpy()), 4))
        self.validation_step_outputs.clear()
        self.validation_auroc.clear()
        self.validation_acc.clear()
    
    def test_step(self, batch, batch_index):
        loss, out, y = self._common_step(batch, batch_index)
        pred = self.softmax(out)
        t_auroc = auroc(pred, y, task = 'multiclass', num_classes = self.config['n_labels'])
        t_acc = accuracy(pred, y, task = 'multiclass', num_classes = self.config['n_labels'])
        self.test_step_outputs.append(loss)
        self.test_auroc.append(t_auroc)
        self.test_acc.append(t_acc)
        self.log("Test Accuracy", t_acc, prog_bar = True, logger = True)
        return loss

    def predict_step(self, batch, batch_index):
        loss, out, y = self._common_step(batch, batch_index)
        return loss
    
    def _common_step(self, batch, batch_index):
        img = batch['image']
        text = batch['input_ids']
        attn_mask = batch['attention_mask']
        y = batch['labels']
        y = y.type(torch.LongTensor)
        y = y.to(config['device'])
        img = img.type(torch.cuda.FloatTensor)
        out = self.forward(img, text, attn_mask)
        loss = self.loss(out, y)
        return loss, out, y
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr = self.config['lr'], weight_decay = config['w_decay'])
        return [optimizer]
    
    def plot_loss(self):
        self.vloss.pop()
        plt.plot(self.tloss, label = 'Training')
        plt.plot(self.vloss, label = 'Validation')
        plt.title('Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()
        
    def plot_auroc(self):
        self.vauroc.pop()
        plt.plot(self.tauroc, label = 'Training')
        plt.plot(self.vauroc, label = 'Validation')
        print('Training AUROC')
        print(self.tauroc)
        print('Validation AUROC')
        print(self.vauroc)
        plt.title('AUROC')
        plt.xlabel('Epochs')
        plt.ylabel('AUROC')
        plt.legend()
        plt.show()
        
    def plot_accuracy(self):
        self.vacc.pop()
        plt.plot(self.tacc, label = 'Training')
        plt.plot(self.vacc, label = 'Validation')
        print('Training Accuracy')
        print(self.tacc)
        print('Validation Accuracy')
        print(self.vacc)
        plt.title('Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()

In [ ]:
checkpoint_callback = ModelCheckpoint(
        dirpath = config['ckpt_path'], 
        filename = "Finetuned_Hateful_CKPT",
        every_n_epochs = config['val_every'],
        save_top_k = 1,
        monitor = 'Validation Accuracy',
        mode = 'max'
    )

In [ ]:
# datamodule

if config['reload'] == True:
    memes = Load_Memes(config)
    memes_data_module = Memes_Data_Module(*memes, batch_size = config['batch_size'], text_model = config['tmodel_name'])
else:
    train_image = torch.load(config['img_train_rl'])
    val_image = torch.load(config['img_val_rl'])
    test_image = torch.load(config['img_test_rl'])
    with open(config['label_train_rl'], 'rb') as f:
        train_labels  = pickle.load(f)
    with open(config['label_val_rl'], 'rb') as f:
        val_labels = pickle.load(f)
    with open(config['label_test_rl'], 'rb') as f:
        test_labels = pickle.load(f)
    with open(config['text_train_rl'], 'rb') as f:
        train_text  = pickle.load(f)
    with open(config['text_val_rl'], 'rb') as f:
        val_text = pickle.load(f)
    with open(config['text_test_rl'], 'rb') as f:
        test_text = pickle.load(f)
    memes_data_module = Memes_Data_Module(train_image, train_text, train_labels, val_image, val_text, val_labels,
                                          test_image, test_text, test_labels,
                                          batch_size = config['batch_size'], text_model = config['tmodel_name'])

In [8]:
# model
model = CVT_Roberta(config)

In [ ]:
# trainer and fit
trainer = lp.Trainer(max_epochs = config['n_epochs'], devices = 1, accelerator = "gpu", num_sanity_val_steps = 0,
                    callbacks = [checkpoint_callback], default_root_dir = config['ckpt_path'],
                    check_val_every_n_epoch = config['val_every'])
trainer.fit(model, memes_data_module)
trainer.validate(model, memes_data_module)
trainer.test(model, memes_data_module)

In [ ]:
model.plot_loss()

In [ ]:
model.plot_auroc()

In [ ]:
model.plot_accuracy()

In [28]:
state_dict = torch.load(r'C:\Users\rabby\CS 7643 - Deep Learning\Project\Unimodal Models\Hateful_6465_6695\Finetuned_Hateful_CKPT.ckpt')['state_dict']
model.load_state_dict(state_dict)
del state_dict
model.to(config['device'])

CVT_Roberta(
  (cvt): CVT_Fairface(
    (model): CvtForImageClassification(
      (cvt): CvtModel(
        (encoder): CvtEncoder(
          (stages): ModuleList(
            (0): CvtStage(
              (embedding): CvtEmbeddings(
                (convolution_embeddings): CvtConvEmbeddings(
                  (projection): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(2, 2))
                  (normalization): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
                )
                (dropout): Dropout(p=0.0, inplace=False)
              )
              (layers): Sequential(
                (0): CvtLayer(
                  (attention): CvtAttention(
                    (attention): CvtSelfAttention(
                      (convolution_projection_query): CvtSelfAttentionProjection(
                        (convolution_projection): CvtSelfAttentionConvProjection(
                          (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)

In [29]:
memes = Load_Memes(config)
memes_data_module = Memes_Data_Module(*memes, batch_size = config['batch_size'], text_model = config['tmodel_name'])
memes_data_module.setup()

C:\ProgramData\anaconda3\Lib\site-packages\transformers\models\convnext\feature_extraction_convnext.py:28: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(
C:\Users\rabby\AppData\Local\Temp\ipykernel_19428\3864625937.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img_train = torch.tensor(extractor(torchvision.io.read_image(img_path), return_tensors="pt")['pixel_values'][0])
C:\Users\rabby\AppData\Local\Temp\ipykernel_19428\3864625937.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img_val = torch.tensor(extractor(torchvision.io.read_image(img_path), retu

Training Image 0 loaded.
Training Text 1 loaded.
Training Label 1 loaded.
Validation Image 0 loaded.
Validation Text 1 loaded.
Validation Label 1 loaded.
Test Image 0 loaded.
Test Image 1000 loaded.
Test Text 1000 loaded.
Test Label 1000 loaded.


In [44]:
acc = []
auc = []
for i in range(100):
    acc_list = []
    auroc_list = []
    softmax = nn.Softmax(dim = 1)
    with torch.no_grad():
        for i, j in enumerate(memes_data_module.test_dataloader()):
            j['image'] = j['image'].to(config['device'])
            j['input_ids'] = j['input_ids'].to(config['device'])
            j['attention_mask'] = j['attention_mask'].to(config['device'])
            j['labels'] = j['labels'].to(config['device'])
            pred = softmax(model(j['image'], j['input_ids'], j['attention_mask']))
            temp_acc = accuracy(pred, j['labels'], task = 'multiclass', num_classes = config['n_labels'])
            temp_auroc = auroc(pred, j['labels'], task = 'multiclass', num_classes = config['n_labels'])
            acc_list.append(temp_acc.item())
            auroc_list.append(temp_auroc.item())
    acc_list = np.array(acc_list)
    acc_mean = acc_list.mean().round(4)
    acc.append(acc_mean)
    auc_list = np.array(auroc_list)
    auc_mean = auc_list.mean().round(4)
    auc.append(auc_mean)
acc = np.array(acc)
auc = np.array(auc)

In [47]:
acc

array([0.6289, 0.6162, 0.6201, 0.6123, 0.6396, 0.6289, 0.6309, 0.6318,
       0.6289, 0.6318, 0.6504, 0.6348, 0.6367, 0.6348, 0.6289, 0.6436,
       0.6465, 0.6318, 0.6279, 0.6309, 0.6338, 0.6318, 0.6387, 0.6523,
       0.624 , 0.6396, 0.6309, 0.6133, 0.6328, 0.6406, 0.6445, 0.6279,
       0.6396, 0.6406, 0.6279, 0.624 , 0.6504, 0.6289, 0.6279, 0.625 ,
       0.6299, 0.6279, 0.6406, 0.626 , 0.6338, 0.6191, 0.6396, 0.6289,
       0.6152, 0.6484, 0.6387, 0.6133, 0.6201, 0.623 , 0.625 , 0.6318,
       0.626 , 0.623 , 0.6221, 0.627 , 0.6436, 0.623 , 0.6338, 0.6211,
       0.6328, 0.6357, 0.6406, 0.6377, 0.627 , 0.6348, 0.6357, 0.627 ,
       0.6396, 0.6289, 0.6299, 0.6289, 0.6338, 0.6367, 0.6455, 0.6416,
       0.6357, 0.6523, 0.6514, 0.625 , 0.627 , 0.626 , 0.6357, 0.6309,
       0.6377, 0.6318, 0.6123, 0.6338, 0.626 , 0.6348, 0.6465, 0.6377,
       0.627 , 0.6455, 0.6338, 0.6309])

In [57]:
print('Accuracy Mean:')
print(acc.mean().round(4))
print('Accuracy Std Dev:')
print(acc.std().round(4))
print('-----------------')
print('AUROC Mean:')
print(auc.mean().round(4))
print('AUROC Std Dev:')
print(auc.std().round(4))

Accuracy Mean:
0.6323
Accuracy Std Dev:
0.0089
-----------------
AUROC Mean:
0.716
AUROC Std Dev:
0.0074
